In [1]:
import numpy as np
from nltk.tokenize import  word_tokenize 
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [2]:
data = pd.read_csv("reviews.csv")

data.head()

,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,"Great music service, the audio is high quality...",5,2,NaN
1,2022-07-09 14:21:22,Please ignore previous negative rating. This a...,5,1,NaN
2,2022-07-09 13:27:32,"This pop-up ""Get the best Spotify experience o...",4,0,NaN
3,2022-07-09 13:26:45,Really buggy and terrible to use as of recently,1,1,NaN
4,2022-07-09 13:20:49,Dear Spotify why do I get songs that I didn't ...,1,1,NaN


In [3]:
data["Rating"].value_counts()

Rating
5    22095
1    17653
4     7842
2     7118
3     6886
Name: count, dtype: int64

In [4]:
data.drop(columns=["Time_submitted", "Total_thumbsup", "Reply"], inplace=True)

data.head()

,Review,Rating
0,"Great music service, the audio is high quality...",5
1,Please ignore previous negative rating. This a...,5
2,"This pop-up ""Get the best Spotify experience o...",4
3,Really buggy and terrible to use as of recently,1
4,Dear Spotify why do I get songs that I didn't ...,1


In [5]:
data = data[(data["Rating"] == 5) | (data["Rating"] == 1)]

data


,Review,Rating
0,"Great music service, the audio is high quality...",5
1,Please ignore previous negative rating. This a...,5
3,Really buggy and terrible to use as of recently,1
4,Dear Spotify why do I get songs that I didn't ...,1
6,I love the selection and the lyrics are provid...,5
...,...,...
61586,One day I was able to switch between songs and...,1
61587,It was my favourite app. I feel sorry for arti...,1
61588,Back to one frkng star. First of all there's t...,1
61589,Even though it was communicated that lyrics fe...,1


In [6]:
labels = ["Positif" if i==5 else "Negatif" for i in data["Rating"]]

data.loc[:, "labels"] = labels
data.head()

,Review,Rating,labels
0,"Great music service, the audio is high quality...",5,Positif
1,Please ignore previous negative rating. This a...,5,Positif
3,Really buggy and terrible to use as of recently,1,Negatif
4,Dear Spotify why do I get songs that I didn't ...,1,Negatif
6,I love the selection and the lyrics are provid...,5,Positif


In [7]:
data.shape

(39748, 3)

In [8]:
data[data.duplicated()]

,Review,Rating,labels
2495,Good app for songs,5,Positif
2996,I love Spotify.,5,Positif
3008,Too many ads,1,Negatif
3261,Amazing music app,5,Positif
3308,Too many ads,1,Negatif
...,...,...,...
40690,Great selection of music.,5,Positif
49117,Great selection of music.,5,Positif
49915,Great selection of music.,5,Positif
58543,Great selection of music and podcasts,5,Positif


In [9]:
data = data.drop_duplicates()

data[data.duplicated()]

,Review,Rating,labels


In [10]:
data.isna().sum()

Review    0
Rating    0
labels    0
dtype: int64

In [11]:
data.isnull().sum()

Review    0
Rating    0
labels    0
dtype: int64

Bagaimana caranya untuk melihat data outlier pada kasus text mining ? 

In [12]:
data.head()

,Review,Rating,labels
0,"Great music service, the audio is high quality...",5,Positif
1,Please ignore previous negative rating. This a...,5,Positif
3,Really buggy and terrible to use as of recently,1,Negatif
4,Dear Spotify why do I get songs that I didn't ...,1,Negatif
6,I love the selection and the lyrics are provid...,5,Positif


In [13]:
data.shape

(39567, 3)

In [14]:
def cleaning_text(text):
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?://\S+', '', text)

    return text

data["Review"] = data["Review"].apply(cleaning_text).str.lower()

data.head()

C:\Users\Rizal\AppData\Local\Temp\ipykernel_36856\2884839563.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Review"] = data["Review"].apply(cleaning_text).str.lower()


,Review,Rating,labels
0,"great music service, the audio is high quality...",5,Positif
1,please ignore previous negative rating. this a...,5,Positif
3,really buggy and terrible to use as of recently,1,Negatif
4,dear spotify why do i get songs that i didn't ...,1,Negatif
6,i love the selection and the lyrics are provid...,5,Positif


In [15]:
data.iloc[329].Review

'ads are more i am not interested in this app i am deleting it'

apakah normalisasi perlu ?

In [16]:
tokenized = data["Review"].apply(lambda x: word_tokenize(x))

tokenized

0        [great, music, service, ,, the, audio, is, hig...
1        [please, ignore, previous, negative, rating, ....
3        [really, buggy, and, terrible, to, use, as, of...
4        [dear, spotify, why, do, i, get, songs, that, ...
6        [i, love, the, selection, and, the, lyrics, ar...
                               ...                        
61586    [one, day, i, was, able, to, switch, between, ...
61587    [it, was, my, favourite, app, ., i, feel, sorr...
61588    [back, to, one, frkng, star, ., first, of, all...
61589    [even, though, it, was, communicated, that, ly...
61590    [use, to, be, sooo, good, back, when, i, had, ...
Name: Review, Length: 39567, dtype: object

In [17]:
stops = set(stopwords.words('english'))

def stopword(text):
    text = [word for word in text if word not in stops]
    return text

tokenized = tokenized.apply(lambda x: stopword(x))

tokenized

0        [great, music, service, ,, audio, high, qualit...
1        [please, ignore, previous, negative, rating, ....
3                 [really, buggy, terrible, use, recently]
4        [dear, spotify, get, songs, n't, put, playlist...
6        [love, selection, lyrics, provided, song, 're,...
                               ...                        
61586    [one, day, able, switch, songs, fast, forward,...
61587    [favourite, app, ., feel, sorry, artists, lost...
61588    [back, one, frkng, star, ., first, 's, ``, spo...
61589    [even, though, communicated, lyrics, feature, ...
61590    [use, sooo, good, back, ,, downloaded, (, free...
Name: Review, Length: 39567, dtype: object

In [18]:
stemmer = PorterStemmer()

def stemming(text):
    text = [stemmer.stem(token) for token in text]
    
    return text

stemmed_token = tokenized.apply(lambda x: stemming(x))

stemmed_token

0        [great, music, servic, ,, audio, high, qualiti...
1        [pleas, ignor, previou, neg, rate, ., app, sup...
3                    [realli, buggi, terribl, use, recent]
4        [dear, spotifi, get, song, n't, put, playlist,...
6        [love, select, lyric, provid, song, 're, liste...
                               ...                        
61586    [one, day, abl, switch, song, fast, forward, g...
61587    [favourit, app, ., feel, sorri, artist, lost, ...
61588    [back, one, frkng, star, ., first, 's, ``, spo...
61589    [even, though, commun, lyric, featur, avail, u...
61590    [use, sooo, good, back, ,, download, (, free, ...
Name: Review, Length: 39567, dtype: object

In [19]:

sentences = []
word_set = []
 
for sent in stemmed_token:
    x = [i for i in sent if i.isalpha()]
    sentences.append(x)
    for word in x:
        if word not in word_set:
            word_set.append(word)
 
#Set of vocab 
word_set = set(word_set)
#Total documents in our corpus
total_documents = len(sentences)
 
#Creating an index for each word in our vocab.
index_dict = {} #Dictionary to store index for each word
i = 0
for word in word_set:
    index_dict[word] = i
    i += 1


In [20]:
def count_dict(sentences):
    word_count = {}
    for word in word_set:
        word_count[word] = 0
        for sent in sentences:
            if word in sent:
                word_count[word] += 1
    return word_count
 
word_count = count_dict(sentences)

In [21]:
#Term Frequency
def termfreq(document, word):
    N = len(document)
    occurance = len([token for token in document if token == word])
    return occurance/N

In [22]:
def inverse_doc_freq(word):
    try:
        word_occurance = word_count[word] + 1
    except:
        word_occurance = 1
    return np.log(total_documents/word_occurance)

In [23]:
def tf_idf(sentence):
    tf_idf_vec = np.zeros((len(word_set),))
    for word in sentence:
        tf = termfreq(sentence,word)
        idf = inverse_doc_freq(word)
         
        value = tf*idf
        tf_idf_vec[index_dict[word]] = value 
    return tf_idf_vec

In [24]:
len(word_set)

12895

In [25]:
#TF-IDF Encoded text corpus
vectors = []
for sent in sentences:
    vec = tf_idf(sent)
    vectors.append(vec)
 
pd.DataFrame(vectors)
# vectors

MemoryError: 

In [31]:
len(vectors[1])

12895